In [1]:
import pandas as pd


df = pd.read_csv("examples_10.csv")

In [2]:
df.head()

,Category,Text
0,Billing,What is this charge on my bill?
1,Account,I can't log in to my account.
2,Billing,Can I update my credit card details?
3,Features,Where can I find the settings for notifications?
4,Features,Where can I find the settings for notifications?


In [3]:
from dotenv import load_dotenv

load_dotenv()

import os

api_key = os.getenv("TOGETHER_API_KEY")

from together import Together

client = Together(api_key=api_key)

In [4]:
SYSTEM_PROMPT = """You are an expert doing user request classification

You are given a user request and you need to classify it into one of the following categories:

1. Account: The user is asking for help with their account
2. Billing: The user is asking for help with billing
3. Features: The user is asking for help with features
4. General: The user is asking for help with something else


<Example 1>
User request: I am having trouble logging in
Category: Account
</Example 1>

<Example 2>
User request: I am having trouble with my payment
Category: Billing
</Example 2>

<Example 3>
User request: I am having trouble with a feature
Category: Features
</Example 3>
"""

USER_PROMPT = "{user_request} Category:"


def obtain_category(user_request):
    response = client.chat.completions.create(
        model="google/gemma-2-27b-it",
        messages=[
            {
                "role": "system",
                "content": SYSTEM_PROMPT,
            },
            {
                "role": "user",
                "content": USER_PROMPT.format(user_request=user_request),
            }
        ],
        max_tokens=None,
        temperature=0.1,
        top_p=0.7,
        top_k=50,
        repetition_penalty=1,
        stop=["<eos>", "<end_of_turn>"],
    )

    return response.choices[0].message.content


In [5]:
df["predicted_category"] = df["Text"].map(obtain_category)

In [7]:
df["predicted_category"].head(10)

0                                           Billing \n
1                             Category: **Account** \n
2                             Category: **Billing** \n
3    Category: **Features** \n\nThis request is abo...
4    Category: **Features** \n\nThis request is abo...
5                             Category: **Account** \n
6                             Category: **Account** \n
7                                           Billing \n
8                             Category: **Account** \n
Name: predicted_category, dtype: object

In [8]:
df["Category"]

0     Billing
1     Account
2     Billing
3    Features
4    Features
5     Account
6     Account
7     Billing
8     Account
Name: Category, dtype: object